In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.data import Dataset
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

/Users/paulmatus/opt/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
%%time
df_sub = pd.read_csv( 'sample_submission.csv')
df_train = pd.read_csv( 'train.csv')
df_test = pd.read_csv( 'test.csv')

<string>:2: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 3.77 s, sys: 621 ms, total: 4.39 s
Wall time: 4.66 s


<string>:2: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857895 entries, 0 to 857894
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         857895 non-null  float64
 1   verified        857895 non-null  bool   
 2   reviewTime      857895 non-null  object 
 3   asin            857895 non-null  object 
 4   reviewerName    857719 non-null  object 
 5   reviewText      857616 non-null  object 
 6   summary         857725 non-null  object 
 7   unixReviewTime  857895 non-null  int64  
 8   vote            118740 non-null  object 
 9   style           444406 non-null  object 
 10  image           7142 non-null    object 
 11  userid          857895 non-null  int64  
 12  itemid          857895 non-null  int64  
 13  rating          857895 non-null  float64
dtypes: bool(1), float64(2), int64(3), object(8)
memory usage: 85.9+ MB


In [4]:
# удаление ВСЕХ дублирующих значений:
df_train.drop_duplicates(inplace=True)

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 828935 entries, 0 to 857894
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         828935 non-null  float64
 1   verified        828935 non-null  bool   
 2   reviewTime      828935 non-null  object 
 3   asin            828935 non-null  object 
 4   reviewerName    828771 non-null  object 
 5   reviewText      828661 non-null  object 
 6   summary         828769 non-null  object 
 7   unixReviewTime  828935 non-null  int64  
 8   vote            115991 non-null  object 
 9   style           429265 non-null  object 
 10  image           7026 non-null    object 
 11  userid          828935 non-null  int64  
 12  itemid          828935 non-null  int64  
 13  rating          828935 non-null  float64
dtypes: bool(1), float64(2), int64(3), object(8)
memory usage: 89.3+ MB


In [6]:
df_train

,overall,verified,reviewTime,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,userid,itemid,rating
0,5.0,True,"10 4, 2016",B01CPNIEQG,Heather,These are my FAVORITE spices in my collection....,Must Add to your Spice kitchen!,1475539200,NaN,NaN,NaN,102179,37138,1.0
1,5.0,True,"03 1, 2016",B006F63M8U,Linda Odom,Add A package to my Coffee and it makes a good...,Milk Chocolate Swiis MIss Hot Cocoa Mix,1456790400,NaN,{'Size:': ' 60-0.73 oz Envelopes'},NaN,3625,17322,1.0
2,5.0,True,"06 26, 2016",B00112O8NG,DesertBlossom,"I love the Torani syrups, but not the prices o...","Love these sugar free syrups, but didn't love ...",1466899200,28,NaN,NaN,39495,5600,1.0
3,5.0,True,"09 5, 2013",B00MRAQUM8,Connie L. Reaves,Very fragrant. The price can't be beat. I do l...,Nice !,1378339200,NaN,NaN,NaN,72854,30249,1.0
4,5.0,True,"09 14, 2016",B000R7RN0G,Mike P.,These are my favorite jalapeno chips so far. T...,These are my favorite jalapeno chips so far,1473811200,NaN,{'Flavor:': ' Jalapeo'},NaN,74859,4349,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857889,5.0,True,"11 4, 2016",B008QMX2SG,B. M. C,Extra good and large,So good,1478217600,NaN,NaN,NaN,70292,24556,1.0
857890,5.0,True,"04 2, 2013",B000H7ELTW,chezlouise,Next to the round variety hanging on my cherry...,next to the round variety............,1364860800,NaN,{'Flavor:': ' Dried Cherries'},NaN,59433,2566,1.0
857892,4.0,False,"09 21, 2016",B000LKX6R8,bamboo grove,"I really like these noodles, I just gave it on...",good but better value elsewhere,1474416000,NaN,{'Size:': ' Pack of 8'},NaN,88,3251,1.0
857893,4.0,True,"11 1, 2015",B00BUKL666,Speedway91,"I like these, they are the kind for a snack an...",Great selection,1446336000,NaN,NaN,NaN,41540,22208,1.0


In [7]:
df_train[(df_train.userid == 538)].rating.value_counts()

0.0    46
1.0    43
Name: rating, dtype: int64

In [8]:
df_test.userid.max()

127495

In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285965 entries, 0 to 285964
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   verified        285965 non-null  bool  
 1   reviewTime      285965 non-null  object
 2   asin            285965 non-null  object
 3   reviewerName    285918 non-null  object
 4   unixReviewTime  285965 non-null  int64 
 5   vote            39462 non-null   object
 6   style           147680 non-null  object
 7   image           2368 non-null    object
 8   userid          285965 non-null  int64 
 9   itemid          285965 non-null  int64 
 10  Id              285965 non-null  int64 
dtypes: bool(1), int64(4), object(6)
memory usage: 22.1+ MB


In [10]:
df_train_train, df_train_test = train_test_split(df_train, random_state=32, shuffle=True)

In [11]:
ratings_coo = sparse.coo_matrix((df_train['rating'].astype(int),
                                 (df_train['userid'],
                                  df_train['itemid'])))

In [12]:
ratings_coo.row

array([102179,   3625,  39495, ...,     88,  41540,    466], dtype=int32)

In [13]:
df_train.rating.value_counts()

1.0    703165
0.0    125770
Name: rating, dtype: int64

In [14]:
df_train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207234 entries, 794187 to 53126
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         207234 non-null  float64
 1   verified        207234 non-null  bool   
 2   reviewTime      207234 non-null  object 
 3   asin            207234 non-null  object 
 4   reviewerName    207191 non-null  object 
 5   reviewText      207176 non-null  object 
 6   summary         207197 non-null  object 
 7   unixReviewTime  207234 non-null  int64  
 8   vote            29018 non-null   object 
 9   style           107065 non-null  object 
 10  image           1749 non-null    object 
 11  userid          207234 non-null  int64  
 12  itemid          207234 non-null  int64  
 13  rating          207234 non-null  float64
dtypes: bool(1), float64(2), int64(3), object(8)
memory usage: 22.3+ MB


In [15]:
ratings_coo.shape

(127496, 41320)

In [16]:
%%time
model = LightFM(learning_rate=0.089, 
                loss='logistic', 
                no_components=160,  
                learning_schedule='adagrad',
               random_state = 20) 
model = model.fit(ratings_coo, epochs=23,   
                  num_threads=12)

CPU times: user 1min 15s, sys: 531 ms, total: 1min 16s
Wall time: 1min 17s


In [17]:
%%time
preds = model.predict(df_train_test.userid.values,
                      df_train_test.itemid.values)
sklearn.metrics.roc_auc_score(df_train_test.rating, preds)

CPU times: user 165 ms, sys: 16.8 ms, total: 182 ms
Wall time: 188 ms


0.9990486042258612

In [18]:
model.get_user_representations()[0][0:5]

array([0.47720245, 0.65928954, 0.89868677, 0.32016027, 0.5377732 ],
      dtype=float32)

In [19]:
model.predict(1, [0,1,2,3])

array([2.13378024, 1.45184779, 2.01253343, 2.56492519])

In [20]:
preds = model.predict(df_train_test.userid.values,
                      df_train_test.itemid.values,
                     )

In [21]:
roc_auc_score(df_train_test.rating, preds)

0.9990486042258612

In [22]:
preds_sub = model.predict(df_test.userid.values,
                      df_test.itemid.values)

In [23]:
preds_sub.min(), preds_sub.max()

(-34.076595306396484, 33.06547164916992)

In [24]:
normalized_preds = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()

In [25]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [26]:
df_sub['rating']= normalized_preds

In [51]:
df_sub.to_csv('submission_base_log94.csv', index=False)

In [52]:
#0.76401

In [ ]:
# result - 0.76820

In [ ]:
#  Модель нейросетей на TensorFlow

In [11]:
# Install TensorFlow

import tensorflow as tf